In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
data.drop('id', axis = 1, inplace = True)
data.head()

In [ ]:
og_data = pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')
og_data.head()


In [ ]:
data =pd.concat([data,og_data])


In [ ]:
data.head()


In [ ]:
data.info()

In [ ]:
data["Gender"] = data["Gender"].map({"Male": 0,
        "Female":1})

In [ ]:
data['NObeyesdad'].unique()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
plt.hist(data['Age'])

In [ ]:
num_col = ["Age","Height","FCVC","NCP","CH2O","FAF","TUE"]

for i in num_col:
    plt.hist(data[i])
    plt.show()

In [ ]:
data['SMOKE'].unique()

In [ ]:
data.columns


In [ ]:
cat_col = ['family_history_with_overweight','FAVC',"CAEC","SMOKE",'SCC',"CALC","MTRANS"]

for i in cat_col:
    plt.scatter(data["NObeyesdad"],data[i], label = cat_col)
    plt.xlabel('NObeyesdad')
    plt.ylabel(i)
    plt.legend()

    # Show the plot
    plt.show()



In [ ]:
yes_no_cols = ['family_history_with_overweight','FAVC',"SMOKE",'SCC']

for i in yes_no_cols:
               data[i] = data[i].map({"yes": 0,
        "no":1})

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data["CAEC"].unique()

In [ ]:
data["CAEC"] = data["CAEC"].map({"no": 0,
        "Sometimes":1,
        "Frequently":2,
        "Always":3})

In [ ]:
data['CALC'].unique()

In [ ]:
data["CALC"] = data["CALC"].map({"no": 0,
        "Sometimes":1,
        "Frequently":2})

In [ ]:
data["MTRANS"].unique()

In [ ]:
data["MTRANS"] = data["MTRANS"].map({"Public_Transportation": 0,
        "Automobile":1,
        "Walking":2,
        "Motorbike":3,
        "Bike":4})

In [ ]:
data.head()

In [ ]:
data["NObeyesdad"].unique()

In [ ]:
data["NObeyesdad"] = data["NObeyesdad"].map({"Insufficient_Weight": 0,
                                            "Normal_Weight":1,
                                            "Overweight_Level_I":2,
                                            "Overweight_Level_II":3,
                                            "Obesity_Type_I":4,
                                            "Obesity_Type_II":5,
                                            "Obesity_Type_III":6})

In [ ]:
data.head()

In [ ]:
test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')
test.head()

In [ ]:
test['CALC'].unique()

In [ ]:
test.drop('id', axis = 1, inplace = True)

yes_no_cols = ['family_history_with_overweight',"SMOKE",'FAVC','SCC']

for i in yes_no_cols:
               test[i] = test[i].map({"yes": 0,
        "no":1})
        

        
test["CAEC"] = test["CAEC"].map({"no": 0,
        "Sometimes":1,
        "Frequently":2,
        "Always":3})

test["CALC"] = test["CALC"].map({"no": 0,
        "Sometimes":1,
        "Frequently":2,
        "Always":3})

test["MTRANS"] = test["MTRANS"].map({"Public_Transportation": 0,
        "Automobile":1,
        "Walking":2,
        "Motorbike":3,
        "Bike":4})

test["Gender"] = test["Gender"].map({"Male": 0,
        "Female":1})

In [ ]:
test.head()

In [ ]:
test_id = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
test_id = test_id['id']
pd.DataFrame(test_id)
test_id.head()

In [ ]:
sample.head()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = data.drop(['NObeyesdad'],axis=1)
y = data['NObeyesdad']


# Instantiate the XGBoost classifier
model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model on the training set
model.fit(X, y)

# Make predictions on the test set
xgb_pred = model.predict(test)



In [ ]:
pd.DataFrame(xgb_pred)
print(xgb_pred)



xgb_data= pd.DataFrame(xgb_pred, columns=['NObeyesdad'])
xgb_data.insert(0, 'id', test_id)
xgb_data["NObeyesdad"] = xgb_data["NObeyesdad"].map({0: "Insufficient_Weight",
                                            1: "Normal_Weight",
                                            2:"Overweight_Level_I",
                                            3:"Overweight_Level_II",
                                            4:"Obesity_Type_I",
                                            5:"Obesity_Type_II",
                                            6:"Obesity_Type_III"})

xgb_data.to_csv('new_xgb.csv',index = False)

In [ ]:
test.head()

In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X, y)
lgb_predictions = lgb_model.predict(test)



In [ ]:
pd.DataFrame(lgb_predictions)


lgb_data= pd.DataFrame(lgb_predictions, columns=['NObeyesdad'])
lgb_data.insert(0, 'id', test_id)
lgb_data["NObeyesdad"] = lgb_data["NObeyesdad"].map({0: "Insufficient_Weight",
                                            1: "Normal_Weight",
                                            2:"Overweight_Level_I",
                                            3:"Overweight_Level_II",
                                            4:"Obesity_Type_I",
                                            5:"Obesity_Type_II",
                                            6:"Obesity_Type_III"})
lgb_data.to_csv('new_lgb.csv',index = False)

In [ ]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier(random_state=42, verbose=0)
catboost_model.fit(X, y)
catboost_predictions = catboost_model.predict(test)




In [ ]:
pd.DataFrame(catboost_predictions)


cat_data= pd.DataFrame(catboost_predictions, columns=['NObeyesdad'])
cat_data.insert(0, 'id', test_id)
cat_data["NObeyesdad"] = cat_data["NObeyesdad"].map({0: "Insufficient_Weight",
                                            1: "Normal_Weight",
                                            2:"Overweight_Level_I",
                                            3:"Overweight_Level_II",
                                            4:"Obesity_Type_I",
                                            5:"Obesity_Type_II",
                                            6:"Obesity_Type_III"})

cat_data.to_csv('new_cat.csv',index = False)

In [ ]:
ensemble_pred = np.column_stack((lgb_predictions, catboost_predictions, xgb_pred))

# Create an ensemble prediction by majority voting
final_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=ensemble_pred)

print(final_pred)

In [ ]:
pd.DataFrame(final_pred)


ens_data= pd.DataFrame(final_pred, columns=['NObeyesdad'])
ens_data.insert(0, 'id', test_id)
ens_data["NObeyesdad"] = ens_data["NObeyesdad"].map({0: "Insufficient_Weight",
                                            1: "Normal_Weight",
                                            2:"Overweight_Level_I",
                                            3:"Overweight_Level_II",
                                            4:"Obesity_Type_I",
                                            5:"Obesity_Type_II",
                                            6:"Obesity_Type_III"})
ens_data.to_csv('new_model_ensemble.csv',index = False)